# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118975e+02     1.473435e+00
 * time: 0.8521320819854736
     1     1.065894e+01     8.732508e-01
 * time: 2.316646099090576
     2    -1.184965e+01     1.043044e+00
 * time: 3.0082900524139404
     3    -3.389070e+01     7.727178e-01
 * time: 4.033903121948242
     4    -4.731671e+01     5.290484e-01
 * time: 5.058297157287598
     5    -5.675624e+01     2.068437e-01
 * time: 6.105632066726685
     6    -5.971220e+01     1.375479e-01
 * time: 6.778401136398315
     7    -6.085714e+01     5.951071e-02
 * time: 7.440672159194946
     8    -6.130859e+01     7.093248e-02
 * time: 8.1340651512146
     9    -6.161382e+01     4.257664e-02
 * time: 8.807123184204102
    10    -6.185104e+01     2.703491e-02
 * time: 9.5004141330719
    11    -6.200830e+01     2.005826e-02
 * time: 10.236149072647095
    12    -6.207100e+01     1.765543e-02
 * time: 10.95821499824524
    13    -6.213783e+01     1.429014e-02
 * time: 11.630908012390137
    14 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557683
    AtomicNonlocal      14.8522650
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336821

    total               -62.261666461135
